***PLEASE READ THE  README FILE BEFORE GOING THROUGH CODE.***

In [14]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import math 
import pandas as pd 
import os

In [15]:
flag=0 #this variable is used to identify whehter the operation on the image is for adding to dataframe or for testing the model

In [16]:
real_path = os.listdir("C:/Users/Rahul/Desktop/real/")
# to get the list of all the files in the folder (path of the real faces) 

In [17]:
fake_path= os.listdir("C:/Users/Rahul/Desktop/fake/")
# to get the list of all the files in the folder (path of the fake faces) 

In [18]:
df = pd.DataFrame(columns='Energy(rgb) Energy(h) Energy(s) Energy(Cr) Energy(Cb) Contrast(rgb) Contrast(h) Contrast(s) Contrast(Cr) Contrast(Cb) Correlation(rgb) Correlation(h) Correlation(s) Correlation(Cr) Correlation(Cb) Homogeneity(rgb) Homogeneity(h) Homogeneity(s) Homogeneity(Cr) Homogeneity(Cb) Type'.split())
df #creating empty dataframe to fill the values from the co-occurrence matrix for the selected features 

,Energy(rgb),Energy(h),Energy(s),Energy(Cr),Energy(Cb),Contrast(rgb),Contrast(h),Contrast(s),Contrast(Cr),Contrast(Cb),...,Correlation(h),Correlation(s),Correlation(Cr),Correlation(Cb),Homogeneity(rgb),Homogeneity(h),Homogeneity(s),Homogeneity(Cr),Homogeneity(Cb),Type


***PLEASE REFER TO THE README FILE TO SEE THE FORMULAS USED.***

In [19]:
def Energy(E,n): #function to calculate the engery factor from the cooccurrence matrix
    e=0
    for i in range(0,n):
        for j in range(0,n):
            e= e + E[i,j]*E[i,j]
    e=math.sqrt(e)
    return e

In [20]:
def contrast(C,n):  #function to calculate the contrast factor from the cooccurrence matrix
    c=0
    for i in range(0,n):
        for j in range(0,n):
            c= c + (i-j)*(i-j)*C[i,j]
    return c

In [21]:
def homogenety(H,n): #function to calculate the homogenety factor from the cooccurrence matrix
    h=0
    for i in range(0,n):
        for j in range(0,n):
            h= h + H[i,j]/(1+ (i-j)*(i-j))
    return h

In [22]:
def corr(Co,n): #function to calculate the correlation factor from the cooccurrence matrix
    co=0
    ui=0
    uj=0
    zi=0
    zj=0
    for i in range(0,n):
        for j in range(0,n):
            ui=ui+ i*Co[i,j]
    
    for i in range(0,n):
        for j in range(0,n):
            uj=uj+ j*Co[i,j]
    
    for i in range(0,n):
        for j in range(0,n):
            zi=zi + Co[i,j]*(i-ui)*(i-ui)
            
    
    zi = math.sqrt(zi)
    
    for i in range(0,n):
        for j in range(0,n):
            zj=zj + Co[i,j]*(j - uj)*(j-uj)
        
    zj = math.sqrt(zj)
    
    for i in range(0,n):
        for j in range(0,n):
            co = co + (Co[i,j]*(i-ui)*(j-uj))/(zi*zj)
            
    return co

In [23]:
def binarize(test): #to reduce the number of bins the pixel values in RGB part is made to either 0 or 1 (value taken from the research paper)
    for x in range(0,128):
        for y in range(0,128):
            if test[x,y] > 0:
                test[x,y]=1
            elif test[x,y] <= 0:
                test[x,y]=0

In [24]:
def trun(test): #to reduce the number of bins the pixel values of H , S ,Cr, Cb the values are truncated between [-2,2] (value taken from the research paper)
    for x in range(0,128):
        for y in range(0,128):
            if test[x,y] >= 2:
                test[x,y]=2
            elif test[x,y] < 2 and test[x,y] > -2:
                test[x,y]=test[x,y]
            elif test[x,y] <= -2:
                test[x,y]=-2

In [25]:
def cooc(I,n): #calculation of the cooccurrance matrix
    A=np.zeros((n,n)) #n is the number of distant values in the img matrix after the binarize and truncation operations. n for RGB is 8 (values from [0,7]). n for rest is 5 (values from [-2,2])
    A1=np.zeros((n,n))
    
    for i in range(0,128): 
        for j in range(0,128):
            if j+1<128:
                A[I[i,j],I[i,j+1]] = A[I[i,j],I[i,j+1]] + 1
            if i+1<128:
                A1[I[i,j],I[i+1,j]] = A1[I[i,j],I[i+1,j]] + 1
    N=0
    N1=0
    
    S=np.zeros((n,n))
    S1=np.zeros((n,n))
    
    B = A.transpose()
    B1 = A1.transpose()
    
    S=A+B
    S1 = A1+B1
    
    for i in range(0,n):
        for j in range(0,n):
            N=N+ S[i,j]
            N1=N1+S1[i,j]
    
    z = (1/N)*S 
    z1 = (1/N1)*S1
    
    return z,z1 #in one function call two co-ocurrence matrix are calculated for both the offsets are mentioned in the paper.

In [26]:
def img_operation(img): #function where the main excecution on the images takes place 
    
    img1 =cv2.resize(img, (128, 128)) #this is size chosen for this program 
    
    im_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB) #coversion from BGR to RGB and dividing it into repective components
    r_img = im_rgb[:, :, 0]
    g_img = im_rgb[:, :, 1]
    b_img = im_rgb[:,:,2]
    
    hsv_img = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV) #conversion from BGR to HSV and dividing it into repective components
    h_img = hsv_img[:, :, 0]
    s_img = hsv_img[:, :, 1]
    v_img = hsv_img[:,:,2]
    
    ycc_img = cv2.cvtColor(img1, cv2.COLOR_BGR2YCrCb) #conversion from BGR to YCrCb and dividing it into repective components
    y_img = ycc_img[:, :, 0]
    cr_img = ycc_img[:, :, 1]
    cb_img = ycc_img[:,:,2]
    
    kernel1 = np.array([[-1],[1]]) #declaring the two kernals required for the operation 
    kernel2 = np.array([[-1,1]])
    
    r_img = r_img.astype(np.int16) #this is done so that the matrix can hold the negetive values that comes from the filter operation 
    g_img = g_img.astype(np.int16)
    b_img = b_img.astype(np.int16)
    
    Rr_img1 = cv2.filter2D(r_img,-1,kernel1) #each component undergoes convolution with both the kernals 
    Rr_img2 = cv2.filter2D(r_img,-1,kernel2)

    Rg_img1 = cv2.filter2D(g_img,-1,kernel1)
    Rg_img2 = cv2.filter2D(g_img,-1,kernel2)

    Rb_img1 = cv2.filter2D(b_img,-1,kernel1)
    Rb_img2 = cv2.filter2D(b_img,-1,kernel2)
    
    binarize(Rr_img1) #calling the function binarize for the both the sets of img components for RGB
    binarize(Rr_img2)

    binarize(Rg_img1)
    binarize(Rg_img2)

    binarize(Rb_img1)
    binarize(Rb_img2)
    
    Rrgb1 = Rr_img1 + Rg_img1*2 + Rb_img1*4 #the final Residual component obtained for both the kernals for RGB
    Rrgb2 = Rr_img2 + Rg_img2*2 + Rb_img2*4
    
    h_img = h_img.astype(np.int16)
    s_img = s_img.astype(np.int16)
    cr_img = cr_img.astype(np.int16)
    cb_img = cb_img.astype(np.int16)
    
    Rh_img1 = cv2.filter2D(h_img,-1,kernel1) #the final Residual component obtained for both the kernals for H,S,Cr,Cb
    Rs_img1 = cv2.filter2D(s_img,-1,kernel1)
    Rcr_img1 = cv2.filter2D(cr_img,-1,kernel1)
    Rcb_img1 = cv2.filter2D(cb_img,-1,kernel1)

    Rh_img2 = cv2.filter2D(h_img,-1,kernel2)
    Rs_img2 = cv2.filter2D(s_img,-1,kernel2)
    Rcr_img2 = cv2.filter2D(cr_img,-1,kernel2)
    Rcb_img2 = cv2.filter2D(cb_img,-1,kernel2)
    
    trun(Rh_img1) #calling the funciton trun for both the sets of img components for H,S,Cr,Cb
    trun(Rs_img1)
    trun(Rcr_img1)
    trun(Rcb_img1)

    trun(Rh_img2)
    trun(Rs_img2)
    trun(Rcr_img2)
    trun(Rcb_img2)
    
    tem1,tem2 = cooc(Rrgb1,8) #2 sets of the cooccurrence matrixes are formed for the two different offsets in each call
    tem3,tem4 = cooc(Rrgb2,8)

    Crgb = 1/4*(tem1+tem2+tem3+tem4) #so in total there are 4 matrixes for each  component , 2 offsets X 2 kernals, we take mean of the 4
    
    tem1,tem2 = cooc(Rh_img1,5)
    tem3,tem4 = cooc(Rh_img2,5)

    Ch = 1/4*(tem1+tem2+tem3+tem4)
    
    tem1,tem2= cooc(Rs_img1,5)
    tem3,tem4= cooc(Rs_img2,5)

    Cs = 1/4*(tem1+tem2+tem3+tem4)
    
    tem1,tem2 = cooc(Rcr_img1,5)
    tem3,tem4 = cooc(Rcr_img2,5)

    Ccr = 1/4*(tem1+tem2+tem3+tem4)
    
    tem1,tem2 = cooc(Rcb_img1,5)
    tem3,tem4 = cooc(Rcb_img2,5)

    Ccb = 1/4*(tem1+tem2+tem3+tem4)
    if flag ==0: #if flag is 0 that means data has to be appended into the DataFrame.which requires the data to be a dictionary
        data = {'Energy(rgb)':Energy(Crgb,8),'Energy(h)':Energy(Ch,5),'Energy(s)':Energy(Cs,5),'Energy(Cr)':Energy(Ccr,5),'Energy(Cb)':Energy(Ccb,5),'Contrast(rgb)':contrast(Crgb,8), 'Contrast(h)':contrast(Ch,5),'Contrast(s)':contrast(Cs,5),'Contrast(Cr)':contrast(Ccr,5),'Contrast(Cb)':contrast(Ccb,5),'Correlation(rgb)':corr(Crgb,8),'Correlation(h)':corr(Ch,5),'Correlation(s)':corr(Cs,5),'Correlation(Cr)':corr(Ccr,5),'Correlation(Cb)':corr(Ccb,5),'Homogeneity(rgb)':homogenety(Crgb,8),'Homogeneity(h)':homogenety(Ch,5),'Homogeneity(s)':homogenety(Cs,5),'Homogeneity(Cr)':homogenety(Ccr,5),'Homogeneity(Cb)':homogenety(Ccb,5)}
    elif flag==1: #if the flag is 1 that means data has to be used for checking. which requires the data to be a 2D matrix of values
        data = [[Energy(Crgb,8),Energy(Ch,5),Energy(Cs,5),Energy(Ccr,5),Energy(Ccb,5),contrast(Crgb,8),contrast(Ch,5),contrast(Cs,5),contrast(Ccr,5),contrast(Ccb,5),corr(Crgb,8),corr(Ch,5),corr(Cs,5),corr(Ccr,5),corr(Ccb,5),homogenety(Crgb,8),homogenety(Ch,5),homogenety(Cs,5),homogenety(Ccr,5),homogenety(Ccb,5)]]
    return data

In [27]:
flag=0
for i in range(len(real_path)):
    img = cv2.imread('C:/Users/Rahul/Desktop/real/'+real_path[i])
    data = img_operation(img)
    data.update({'Type':0}) #the real images are set to type 0 
    df = df.append(data,ignore_index=True)

for i in range(len(fake_path)):
    img = cv2.imread('C:/Users/Rahul/Desktop/fake/'+fake_path[i])
    data = img_operation(img)
    data.update({'Type':1}) #the fake images are set to type 1
    df = df.append(data,ignore_index=True)


In [28]:
df

,Energy(rgb),Energy(h),Energy(s),Energy(Cr),Energy(Cb),Contrast(rgb),Contrast(h),Contrast(s),Contrast(Cr),Contrast(Cb),...,Correlation(h),Correlation(s),Correlation(Cr),Correlation(Cb),Homogeneity(rgb),Homogeneity(h),Homogeneity(s),Homogeneity(Cr),Homogeneity(Cb),Type
0,0.383593,0.283755,0.374419,0.275871,0.267993,21.208046,3.442529,1.478516,2.755798,2.868295,...,0.259163,0.211859,0.354561,0.350543,0.457519,0.592560,0.670892,0.662785,0.649282,0.0
1,0.439826,0.297763,0.355579,0.345086,0.389455,13.514733,3.316591,1.684178,2.715505,3.068159,...,0.233526,0.232557,0.440169,0.407400,0.633169,0.598200,0.687189,0.716446,0.708516,0.0
2,0.457787,0.453894,0.301177,0.539247,0.502738,9.455140,2.820297,2.409710,2.363604,2.642640,...,0.285306,0.346459,0.421681,0.430789,0.665600,0.670076,0.652007,0.781746,0.764037,0.0
3,0.431723,0.361831,0.365191,0.377268,0.337848,18.442021,3.423382,1.292369,2.670798,2.630767,...,0.241407,0.449750,0.442823,0.466545,0.543002,0.620153,0.707993,0.730115,0.720432,0.0
4,0.438405,0.473422,0.319677,0.520689,0.575850,16.611851,2.569144,1.904128,1.987558,2.118372,...,0.310391,0.435516,0.524757,0.461749,0.584266,0.694278,0.677362,0.801297,0.804558,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.437839,0.275243,0.417254,0.322867,0.336996,12.597579,3.382167,1.278759,2.528143,2.932717,...,0.226467,0.283307,0.457215,0.425088,0.642508,0.588787,0.743372,0.722054,0.700127,1.0
996,0.475095,0.452182,0.361569,0.522431,0.552420,10.822727,2.079832,1.556825,1.958354,1.746893,...,0.444376,0.483959,0.556721,0.574285,0.689278,0.719979,0.730111,0.809209,0.822229,1.0
997,0.473576,0.358307,0.365771,0.441636,0.457700,9.670091,2.793768,1.598010,2.168476,2.528190,...,0.314801,0.363191,0.512773,0.515218,0.704788,0.652518,0.720030,0.773385,0.762350,1.0
998,0.372510,0.365851,0.359388,0.354763,0.344041,9.825695,2.836614,1.724902,2.466197,2.358637,...,0.369099,0.302762,0.498865,0.479268,0.622794,0.684674,0.704197,0.753544,0.754612,1.0


In [29]:
df.to_csv('real_fake.csv',index=False) #saving the Dataframe to a csv file for later use 

***THIS IS WHERE THE MODEL TRAINING HAPPENS. SVM IS USED AS THE CLASSIFIER FOR THIS THIS CODE.***

In [30]:
df = pd.read_csv('real_fake.csv') #reading from the csv file 

In [31]:
df

,Energy(rgb),Energy(h),Energy(s),Energy(Cr),Energy(Cb),Contrast(rgb),Contrast(h),Contrast(s),Contrast(Cr),Contrast(Cb),...,Correlation(h),Correlation(s),Correlation(Cr),Correlation(Cb),Homogeneity(rgb),Homogeneity(h),Homogeneity(s),Homogeneity(Cr),Homogeneity(Cb),Type
0,0.383593,0.283755,0.374419,0.275871,0.267993,21.208046,3.442529,1.478516,2.755798,2.868295,...,0.259163,0.211859,0.354561,0.350543,0.457519,0.592560,0.670892,0.662785,0.649282,0.0
1,0.439826,0.297763,0.355579,0.345086,0.389455,13.514733,3.316591,1.684178,2.715505,3.068159,...,0.233526,0.232557,0.440169,0.407400,0.633169,0.598200,0.687189,0.716446,0.708516,0.0
2,0.457787,0.453894,0.301177,0.539247,0.502738,9.455140,2.820297,2.409710,2.363604,2.642640,...,0.285306,0.346459,0.421681,0.430789,0.665600,0.670076,0.652007,0.781746,0.764037,0.0
3,0.431723,0.361831,0.365191,0.377268,0.337848,18.442021,3.423382,1.292369,2.670798,2.630767,...,0.241407,0.449750,0.442823,0.466545,0.543002,0.620153,0.707993,0.730115,0.720432,0.0
4,0.438405,0.473422,0.319677,0.520689,0.575850,16.611851,2.569144,1.904128,1.987558,2.118372,...,0.310391,0.435516,0.524757,0.461749,0.584266,0.694278,0.677362,0.801297,0.804558,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.437839,0.275243,0.417254,0.322867,0.336996,12.597579,3.382167,1.278759,2.528143,2.932717,...,0.226467,0.283307,0.457215,0.425088,0.642508,0.588787,0.743372,0.722054,0.700127,1.0
996,0.475095,0.452182,0.361569,0.522431,0.552420,10.822727,2.079832,1.556825,1.958354,1.746893,...,0.444376,0.483959,0.556721,0.574285,0.689278,0.719979,0.730111,0.809209,0.822229,1.0
997,0.473576,0.358307,0.365771,0.441636,0.457700,9.670091,2.793768,1.598010,2.168476,2.528190,...,0.314801,0.363191,0.512773,0.515218,0.704788,0.652518,0.720030,0.773385,0.762350,1.0
998,0.372510,0.365851,0.359388,0.354763,0.344041,9.825695,2.836614,1.724902,2.466197,2.358637,...,0.369099,0.302762,0.498865,0.479268,0.622794,0.684674,0.704197,0.753544,0.754612,1.0


In [32]:
from sklearn.model_selection import train_test_split

In [33]:
from sklearn.utils import shuffle
df = shuffle(df) #used to shuffle the data in the df as it is arrnged as real and fakes for better results when using train test split
df

,Energy(rgb),Energy(h),Energy(s),Energy(Cr),Energy(Cb),Contrast(rgb),Contrast(h),Contrast(s),Contrast(Cr),Contrast(Cb),...,Correlation(h),Correlation(s),Correlation(Cr),Correlation(Cb),Homogeneity(rgb),Homogeneity(h),Homogeneity(s),Homogeneity(Cr),Homogeneity(Cb),Type
61,0.428238,0.430260,0.351202,0.423046,0.416540,17.280727,3.052534,1.419245,2.334184,2.808163,...,0.299110,0.434702,0.471432,0.443574,0.562955,0.677838,0.698392,0.755985,0.737695,0.0
592,0.397358,0.387737,0.341981,0.448855,0.422721,11.431379,2.687208,1.762934,2.100424,2.514625,...,0.360527,0.457942,0.521308,0.476045,0.626383,0.673483,0.718714,0.785157,0.764342,1.0
962,0.425314,0.364868,0.391100,0.405741,0.458883,11.499062,2.427519,1.354838,1.984775,2.082754,...,0.404544,0.405520,0.549164,0.576108,0.647723,0.683827,0.734154,0.781020,0.795975,1.0
55,0.350431,0.314378,0.365060,0.300145,0.321033,12.914032,3.345565,1.700957,2.773899,2.589998,...,0.285582,0.240251,0.375794,0.434968,0.571627,0.625118,0.696516,0.698414,0.716251,0.0
544,0.462022,0.468655,0.365846,0.441440,0.460620,11.547475,2.736067,1.521838,2.256598,2.195266,...,0.319399,0.410774,0.492964,0.563537,0.667412,0.700067,0.720404,0.772104,0.783871,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,0.395020,0.311389,0.343493,0.381932,0.369532,14.773545,3.109759,1.519639,2.559716,2.421198,...,0.283725,0.438689,0.478846,0.476647,0.562874,0.614914,0.694679,0.741479,0.738588,0.0
410,0.397237,0.280798,0.397845,0.334307,0.323674,17.799105,3.439930,1.264041,2.799812,2.716074,...,0.228451,0.244131,0.440549,0.438678,0.524812,0.581840,0.708387,0.711885,0.711994,0.0
426,0.456637,0.464442,0.307588,0.447628,0.540015,13.756536,3.263011,2.042323,2.412232,2.720734,...,0.258909,0.321423,0.439034,0.416359,0.628427,0.678634,0.664695,0.756813,0.766026,0.0
580,0.416115,0.307882,0.368790,0.367142,0.357827,9.252768,3.022684,1.681410,2.418815,2.539878,...,0.299995,0.270658,0.485868,0.479204,0.681685,0.638299,0.706041,0.754107,0.735469,1.0


In [34]:
df.reset_index(inplace=True, drop=True)
df

,Energy(rgb),Energy(h),Energy(s),Energy(Cr),Energy(Cb),Contrast(rgb),Contrast(h),Contrast(s),Contrast(Cr),Contrast(Cb),...,Correlation(h),Correlation(s),Correlation(Cr),Correlation(Cb),Homogeneity(rgb),Homogeneity(h),Homogeneity(s),Homogeneity(Cr),Homogeneity(Cb),Type
0,0.428238,0.430260,0.351202,0.423046,0.416540,17.280727,3.052534,1.419245,2.334184,2.808163,...,0.299110,0.434702,0.471432,0.443574,0.562955,0.677838,0.698392,0.755985,0.737695,0.0
1,0.397358,0.387737,0.341981,0.448855,0.422721,11.431379,2.687208,1.762934,2.100424,2.514625,...,0.360527,0.457942,0.521308,0.476045,0.626383,0.673483,0.718714,0.785157,0.764342,1.0
2,0.425314,0.364868,0.391100,0.405741,0.458883,11.499062,2.427519,1.354838,1.984775,2.082754,...,0.404544,0.405520,0.549164,0.576108,0.647723,0.683827,0.734154,0.781020,0.795975,1.0
3,0.350431,0.314378,0.365060,0.300145,0.321033,12.914032,3.345565,1.700957,2.773899,2.589998,...,0.285582,0.240251,0.375794,0.434968,0.571627,0.625118,0.696516,0.698414,0.716251,0.0
4,0.462022,0.468655,0.365846,0.441440,0.460620,11.547475,2.736067,1.521838,2.256598,2.195266,...,0.319399,0.410774,0.492964,0.563537,0.667412,0.700067,0.720404,0.772104,0.783871,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.395020,0.311389,0.343493,0.381932,0.369532,14.773545,3.109759,1.519639,2.559716,2.421198,...,0.283725,0.438689,0.478846,0.476647,0.562874,0.614914,0.694679,0.741479,0.738588,0.0
996,0.397237,0.280798,0.397845,0.334307,0.323674,17.799105,3.439930,1.264041,2.799812,2.716074,...,0.228451,0.244131,0.440549,0.438678,0.524812,0.581840,0.708387,0.711885,0.711994,0.0
997,0.456637,0.464442,0.307588,0.447628,0.540015,13.756536,3.263011,2.042323,2.412232,2.720734,...,0.258909,0.321423,0.439034,0.416359,0.628427,0.678634,0.664695,0.756813,0.766026,0.0
998,0.416115,0.307882,0.368790,0.367142,0.357827,9.252768,3.022684,1.681410,2.418815,2.539878,...,0.299995,0.270658,0.485868,0.479204,0.681685,0.638299,0.706041,0.754107,0.735469,1.0


In [35]:
X = df.drop('Type',axis=1) #putting the features into X 
y = df['Type']    #Type column is our target 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [36]:
from sklearn.svm import SVC

In [37]:
svc_model = SVC()

In [38]:
svc_model.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [39]:
predictions = svc_model.predict(X_test)

In [40]:
from sklearn.metrics import classification_report,confusion_matrix

In [41]:
print(confusion_matrix(y_test,predictions))

[[130  22]
 [  1 147]]


In [42]:
print(classification_report(y_test,predictions)) 

              precision    recall  f1-score   support

         0.0       0.99      0.86      0.92       152
         1.0       0.87      0.99      0.93       148

    accuracy                           0.92       300
   macro avg       0.93      0.92      0.92       300
weighted avg       0.93      0.92      0.92       300



In [43]:
from sklearn.model_selection import GridSearchCV #this is done to find the optimal parameters for the best outcome from SVM

In [44]:
param_grid = {'C': [0.01,0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001,0.0001]} 

In [45]:
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=4)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.01, gamma=1 .................................................
[CV] ..................... C=0.01, gamma=1, score=0.921, total=   0.0s
[CV] C=0.01, gamma=1 .................................................
[CV] ..................... C=0.01, gamma=1, score=0.936, total=   0.0s
[CV] C=0.01, gamma=1 .................................................
[CV] ..................... C=0.01, gamma=1, score=0.914, total=   0.0s
[CV] C=0.01, gamma=1 .................................................
[CV] ..................... C=0.01, gamma=1, score=0.936, total=   0.0s
[CV] C=0.01, gamma=1 .................................................
[CV] ..................... C=0.01, gamma=1, score=0.929, total=   0.0s
[CV] C=0.01, gamma=0.1 ...............................................
[CV] ................... C=0.01, gamma=0.1, score=0.900, total=   0.0s
[CV] C=0.01, gamma=0.1 ...............................................
[CV] ..........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s


[CV] ................... C=0.01, gamma=0.1, score=0.943, total=   0.0s
[CV] C=0.01, gamma=0.01 ..............................................
[CV] .................. C=0.01, gamma=0.01, score=0.871, total=   0.0s
[CV] C=0.01, gamma=0.01 ..............................................
[CV] .................. C=0.01, gamma=0.01, score=0.886, total=   0.0s
[CV] C=0.01, gamma=0.01 ..............................................
[CV] .................. C=0.01, gamma=0.01, score=0.843, total=   0.0s
[CV] C=0.01, gamma=0.01 ..............................................
[CV] .................. C=0.01, gamma=0.01, score=0.871, total=   0.0s
[CV] C=0.01, gamma=0.01 ..............................................
[CV] .................. C=0.01, gamma=0.01, score=0.921, total=   0.0s
[CV] C=0.01, gamma=0.001 .............................................
[CV] ................. C=0.01, gamma=0.001, score=0.507, total=   0.0s
[CV] C=0.01, gamma=0.001 .............................................
[CV] .

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:    1.5s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=4)

In [46]:
grid.best_params_

{'C': 100, 'gamma': 0.1}

In [47]:
grid_predictions = grid.predict(X_test)

In [48]:
print(confusion_matrix(y_test,grid_predictions))

[[146   6]
 [  5 143]]


In [49]:
print(classification_report(y_test,grid_predictions))

              precision    recall  f1-score   support

         0.0       0.97      0.96      0.96       152
         1.0       0.96      0.97      0.96       148

    accuracy                           0.96       300
   macro avg       0.96      0.96      0.96       300
weighted avg       0.96      0.96      0.96       300



***As you can see, after applying grid search we got a better accuracy for the model.***


***This furthur section is for testing our model. If it produces proper results for the images not in the data set. You can find the folders named real_test and fake_test which contains the test images. input it below to check whether the model is working properly.***

In [50]:
img = cv2.imread('C:/Users/Rahul/Desktop/fake_test/9.jpg')

In [51]:
flag = 1
data = img_operation(img)

In [52]:
predictions = svc_model.predict(data)

In [53]:
print(predictions)

[1.]


***0 indicates the input image is REAL and 1 indicates the input image is FAKE***